# PQ Recall

In [ ]:
import nanopq
import numpy as np
%cd ..
%cd src
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

## product quantization

In [2]:

n1,n2, D = 10000, 2000, 128
np.random.seed(15)
X = np.random.randn(n1, D).astype(np.float32)  
queries = np.random.randn(n2,D).astype(np.float32)  

# Instantiate with M=8 sub-spaces,Ks=256 codewords in each sub-space
M,Ks=8,256
pq = nanopq.PQ(M=M,Ks=Ks)

# Train codewords
pq.fit(X)

# Encode to PQ-codes
X_code = pq.encode(X)  # (10000, 8) 

M: 8, Ks: 256, code_dtype: <class 'numpy.uint8'>
iter: 20, seed: 123
Training the subspace: 0 / 8
Training the subspace: 1 / 8
Training the subspace: 2 / 8
Training the subspace: 3 / 8
Training the subspace: 4 / 8
Training the subspace: 5 / 8
Training the subspace: 6 / 8
Training the subspace: 7 / 8
Encoding the subspace: 0 / 8
Encoding the subspace: 1 / 8
Encoding the subspace: 2 / 8
Encoding the subspace: 3 / 8
Encoding the subspace: 4 / 8
Encoding the subspace: 5 / 8
Encoding the subspace: 6 / 8
Encoding the subspace: 7 / 8


### compute recall

In [3]:
from evaluationRecall import SearchNeighbors_PQ, recall_atN

# M (int): The number of sub-space
# Ks (int): The number of codewords for each subspace
#     (typically 256, so that each sub-vector is quantized
#     into 256 bits = 1 byte = uint8)
# D (int): The dim of each vector
# pq_codebook (np.ndarray): shape=(M, Ks, Ds) with dtype=np.float32.
#     codebook[m][ks] means ks-th codeword (Ds-dim) for m-th subspace
# pq_codes (np.ndarray): PQ codes with shape=(n, M) and dtype=np.int
# metric (str): dot_product or l2_distance   

rpq = SearchNeighbors_PQ(M=M, Ks=Ks, D=D, pq_codebook = pq.codewords, pq_codes = X_code, metric="l2_distance")

# This will get the true nearest neighbor of the queries by  brute force search.
ground_truth = rpq.brute_force_search(X, queries, metric = "l2_distance") 

In [4]:
# This will get topk neighbors(rpq.neighbors_matrix) of queries and compute the recall
neighbors_matrix = rpq.neighbors(queries,topk = 512)
recall_atN(neighbors_matrix, ground_truth)

neighbors took 1.7038955688476562 seconds
recall 1@1 = 0.0365
recall 1@2 = 0.0685
recall 1@4 = 0.1085
recall 1@8 = 0.168
recall 1@10 = 0.1885
recall 1@16 = 0.238
recall 1@20 = 0.2645
recall 1@32 = 0.329
recall 1@64 = 0.4375
recall 1@100 = 0.528
recall 1@128 = 0.567
recall 1@256 = 0.7105
recall 1@512 = 0.8385


N=[1, 2, 4, 8, 10, 16, 20, 32, 64, 100, 128, 256, 512]
recall1@N:[0.0365, 0.0685, 0.1085, 0.168, 0.1885, 0.238, 0.2645, 0.329, 0.4375, 0.528, 0.567, 0.7105, 0.8385]


# AQ Recall

The following is not additive quantization, only the codebooks and codes have the same structure as additive quantization

In [7]:
import numpy as np
from scipy.cluster.vq import kmeans2

n, nq, D = 10000, 2000, 128
np.random.seed(15)
X = np.random.randn(n, D).astype(np.float32)  
queries = np.random.randn(nq,D).astype(np.float32)
M,K = 8,256

centroid, code = kmeans2(X, K, minit='points')
centroid.shape  # shape = (256,128)

codebooks = centroid
codes = code 
RX = X
for i in range(1,M):
    RX = RX - centroid[code]

    centroid , code = kmeans2(RX, K)

    codebooks = np.r_[codebooks,centroid]
    codes = np.c_[codes,code]
print(codebooks.shape)
print(codes.shape)

(2048, 128)
(10000, 8)


## compute recall

In [9]:
from evaluationRecall import SearchNeighbors_AQ, recall_atN

# M (int): The number of codebooks  
# K (int): The number of codewords for each codebook  
# D (int): The dim of each vector  
# aq_codebooks (np.ndarray): shape=(M*K, D) with dtype=np.float32.  
#     aq_codebooks[0:K,:] represents the K codewords in the first codebook  
#     aq_codebooks[(m-1)*K:mK,:] represents the K codewords in the m-th codebook  
# aq_codes (np.ndarray): AQ codes with shape=(n, M) and dtype=np.int, where n is the number of encoded datapoints.  
    # aq_codes[i,j] is in {0,1,...,K-1} for all i,j
# metric (str): dot_product or l2_distance 

raq = SearchNeighbors_AQ(M = M, K = K, D = D, aq_codebooks = codebooks, aq_codes = codes, metric="l2_distance")

# This will get the true nearest neighbor of the queries by  brute force search.
ground_truth = raq.brute_force_search(X,queries,metric="l2_distance")

In [10]:
# This will get topk neighbors(raq.neighbors_matrix) of queries and compute the recall
neighbors_matrix = raq.neighbors(queries=queries, topk=100)
recall_atN(neighbors_matrix,ground_truth)

recall 1@1 = 0.059
recall 1@2 = 0.0985
recall 1@4 = 0.161
recall 1@8 = 0.2345
recall 1@10 = 0.269
recall 1@16 = 0.335
recall 1@20 = 0.3645
recall 1@32 = 0.4315
recall 1@64 = 0.5395
recall 1@100 = 0.6285


N=[1, 2, 4, 8, 10, 16, 20, 32, 64, 100]
recall1@N:[0.059, 0.0985, 0.161, 0.2345, 0.269, 0.335, 0.3645, 0.4315, 0.5395, 0.6285]
